In [2]:
import sys
import os
sys.path.append("/home/gant/Documents/tpcanalysis/python_comp/src")
from openImage import openImage
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import cm

from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
from wolframclient.serializers import export
from wolframclient.language.expression import WLFunction



def fitImage(session, path_and_name_input, path_and_name_output, draw_opt=True):
    
    
    lines_th = 0.02
    
    # Evaluate the expressions
    session.evaluate('img = Import["{}"];'.format(path_and_name_input))
    session.evaluate('whiteth = 0.3;')
    session.evaluate('bwimage = Binarize[img, whiteth];')
    session.evaluate('bwimage = Thinning[bwimage];')
    session.evaluate('lines = ImageLines[bwimage, {}, Method -> "RANSAC"];'.format(lines_th))
    


    #Draw and export if option is chosen
    if(draw_opt):
        session.evaluate('highlightedimage = HighlightImage[bwimage, lines];')
        session.evaluate('highlightedimageoriginal = HighlightImage[img, lines]')
        #session.evaluate('Export["{}", highlightedimage];'.format(path_and_name_output))
        session.evaluate('Export["{}", highlightedimageoriginal];'.format(path_and_name_output))



    #Calculate slopes and intercepts
    """ session.evaluate('slopes = Table[(lines[[i, 1, 2, 2]] - lines[[i, 1, 1, 2]])/(lines[[i, 1, 2, 1]] - lines[[i, 1, 1, 1]]), {i, 1, Length[lines]}];')
    session.evaluate('intercepts = Table[lines[[i, 1, 1, 2]] - slopes[[i]]*lines[[i, 1, 1, 1]], {i, 1, Length[lines]}];')
    
    slopes = session.evaluate('slopes')
    intercepts = session.evaluate('intercepts')
    equations = list(zip(slopes, intercepts))

    #Change DirectedInfinity[] to float('inf') when the slope is infinite.
    equations = [(slope, intercept) if not isinstance(slope, WLFunction) and not isinstance(intercept, WLFunction) else (float('inf'), float('inf')) for slope, intercept in equations]

    return equations """

    #Simpler to just use the segments and manipulate them in Python.
    #Python is simpler.
    session.evaluate('x1 = Table[lines[[i, 1, 1, 1]], {i, 1, Length[lines]}]')
    session.evaluate('y1 = Table[lines[[i, 1, 1, 2]], {i, 1, Length[lines]}]')
    session.evaluate('x2 = Table[lines[[i, 1, 2, 1]], {i, 1, Length[lines]}]')
    session.evaluate('y2 = Table[lines[[i, 1, 2, 2]], {i, 1, Length[lines]}]')
    x1 = session.evaluate('x1')
    y1 = session.evaluate('y1')
    x2 = session.evaluate('x2')
    y2 = session.evaluate('y2')

    lines = list(zip(zip(x1,y1),zip(x2,y2)))

    return lines








#Make sure the dirs exist befoe calling the function. Don't have time to make sure in code.
image_path = "/media/gant/Expansion/tpc_root_raw/DATA_ROOT/cleanimages2/"

image_list = ["CoBo_2018-06-20T10-51-39.459_0000/25",
              "CoBo_2018-06-20T10-51-39.459_0002/511",
              "CoBo_2018-06-20T10-51-39.459_0002/672",
              "CoBo_2018-06-20T10-51-39.459_0002/1019",
              "CoBo_2018-06-20T10-51-39.459_0003/1319",
              "CoBo_2018-06-20T10-51-39.459_0003/1428",
              "CoBo_2018-06-20T10-51-39.459_0004/114",
              "CoBo_2018-06-20T10-51-39.459_0004/319",
              "CoBo_2018-06-20T10-51-39.459_0004/417",
              "CoBo_2018-06-20T10-51-39.459_0004/448",
              "CoBo_2018-06-20T10-51-39.459_0004/1456",
              "CoBo_2018-06-20T10-51-39.459_0004/1596",
              "CoBo_2018-06-20T10-51-39.459_0004/1725",
              "CoBo_2018-06-20T10-51-39.459_0005/477",
              "CoBo_2018-06-20T10-51-39.459_0005/677",
              "CoBo_2018-06-20T10-51-39.459_0005/1113",
              "CoBo_2018-06-20T10-51-39.459_0005/1190",
              "CoBo_2018-06-20T10-51-39.459_0005/1311",
              "CoBo_2018-06-20T10-51-39.459_0005/1843"]

image_suffix_list = ["_u.png", "_v.png", "_w.png"]


output_path = "/media/gant/Expansion/tpc_root_raw/DATA_ROOT/fittedimages/"

session = WolframLanguageSession()



try:
    # Evaluate the expressions
    """ session.evaluate('img = Import["{}"];'.format("/media/gant/Expansion/tpc_root_raw/DATA_ROOT/cleanimages/CoBo_2018-06-20T10-51-39.459_0000/25_u.png"))
    session.evaluate('whiteth = 0.3;')
    session.evaluate('bwimage = Binarize[img, whiteth];')
    session.evaluate('bwimage = Thinning[bwimage];')
    session.evaluate('lines = ImageLines[bwimage, 0.01, Method -> "RANSAC"];')
    session.evaluate('highlightedimage = HighlightImage[bwimage, lines];')

    # Get the output of the expressions
    output_image = session.evaluate('Image[highlightedimage, ImageSize -> {800, 600}]')



    # Export the output image if a specified output path is provided  
    session.evaluate('Export["{}", highlightedimage];'.format("/media/gant/Expansion/tpc_root_raw/DATA_ROOT/fittedimages/CoBo_2018-06-20T10-51-39.459_0000/25_u.png"))
 """

    for image in image_list:
        for suffix in image_suffix_list:
            output_lines = fitImage(session, (image_path + image + suffix), (output_path + image + suffix), True)
            #print(f"Lines: {output_lines}\n")

finally:
    session.terminate()
